# **Inference_yolo_OBB_to_cvat**

In [16]:
model_path = r"C:\Users\CorneAI\YOLOv11_OBB\models\yolo11n_obb_1-295_1to139.pt"

# すでにアノーテーションされているラベルは変えずに、その次の画像から推論を行う
# CVATの仕様上、2より12や134が優先されることに留意
start_id = 501 #まだinferenceされていない最初のID (xmlから確認)
end_id = 1000 #Inferenceしたい最後のID（xmlから確認）

# すでにアノーテーションされているxmlファイルのパス
orig_xml_path = r"C:\Users\CorneAI\YOLOv11_OBB\eyelid_image\1-295_periocular\1-295obb_1to139_cvat.xml"

# 推論を行ったxmlファイルのパス（すでに行われているアノーテーションは変えない）
output_xml_path = r"C:\Users\CorneAI\YOLOv11_OBB\eyelid_image\1-295_periocular\1-295obb_139to160_cvat.xml"
output_temp_label_dir = r"C:\Users\CorneAI\YOLOv11_OBB\eyelid_image\1-295_periocular\temp_labels"

In [ ]:
import xml.etree.ElementTree as ET
import os

image_paths = []
try:
    # XMLファイルを解析
    tree = ET.parse(orig_xml_path)
    root = tree.getroot()

    # <image> タグをすべて検索
    for image_tag in root.findall('image'):
        # 'name' 属性（通常、画像ファイルパスまたは名前を含む）を取得
        name = image_tag.get('name')
        if name:
            # ここではXML内のパスをそのままリストに追加します。
            # 必要に応じて、ベースディレクトリと結合して絶対パスにしてください。
            # 例: base_image_dir = os.path.dirname(orig_xml_path) # または適切な画像ディレクトリ
            #    full_path = os.path.join(base_image_dir, name)
            #    image_paths.append(full_path)
            image_paths.append(name)
        else:
            print(f"警告: ID {image_tag.get('id')} の画像タグに 'name' 属性が見つかりません。")

    print(f"XMLファイルから {len(image_paths)} 個の画像パスを読み込みました。")
    #print("最初の5件:", image_paths[:5]) # 確認用

except FileNotFoundError:
    print(f"エラー: XMLファイルが見つかりません: {orig_xml_path}")
except ET.ParseError:
    print(f"エラー: XMLファイルの解析に失敗しました: {orig_xml_path}")
except Exception as e:
    print(f"予期せぬエラーが発生しました: {e}")

# image_paths リストが作成されました。
# このリストは、XMLファイルに記載されている順序での画像パス（または名前）を含みます。


XMLファイルから 4467 個の画像パスを読み込みました。
最初の5件: ['1-20141126-38-091804_eb568e2ac952f8be45ec0ac9ae800120b7c988b60ac499ca87306986d218f554_L.jpg', '1-20141126-38-091804_eb568e2ac952f8be45ec0ac9ae800120b7c988b60ac499ca87306986d218f554_R.jpg', '1-20150121-38-142903_6e60b2355e174936406b708cf171e424300de779d4b8ae8e3aebb1a9de9905e6_L.jpg', '1-20150121-38-142903_6e60b2355e174936406b708cf171e424300de779d4b8ae8e3aebb1a9de9905e6_R.jpg', '1-20150121-38-142903_f27c8cf98eef934c557ffb70fed94b3fb9d400f29f472f2687b7fcec96e92c77_L.jpg']
